In [8]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [2]:
env = gym.make('Taxi-v3')

In [3]:
episodes = 4

for episode in range(1,episodes):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        #let us make ou agent take random actions initially
        state, reward, done, info = env.step(env.action_space.sample())
        #This has 6 random acitons we can take.
        #Increment the reward everytime
        score +=  reward
        clear_output(wait=True)
    print('Episode: {}\n Score :{}'.format(episode, score))


env.close()

Episode: 3
 Score :-776


In [6]:
#Creating the Q table for the agent
actions = env.action_space.n
state = env.observation_space.n
state

500

In [9]:
#since we have 500 states, let us build the q table.
q_table = np.zeros((state,actions))

In [10]:
#parametres for Q learning
num_episodes = 10000
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

#this is for the exploration vs exploitation dilemma.
exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

rewards_all_episodes = []


In [13]:
#Now we start training our model.
for episode in range(num_episodes):
    state = env.reset()
    done = False
    rewards_current_episode = 0

    #This is the breakout of condition if our agent falls in a loop.
    for step in range(max_steps_per_episode):
        exploration_threshold = np.random.uniform(0,1)
        #Either take a random action or the max depedning on the params.
        if exploration_threshold > exploration_rate:
            action = np.argmax(q_table[state,:])
        else:
            action = env.action_space.sample()

        #get the values for taking that action.
        new_state, reward, done, info = env.step(action)
        
        #fill in the Q table using the slightly modified version of Q learning update rule.
        q_table[state,action] = q_table[state,action] * (1-learning_rate) +  learning_rate*(reward + discount_rate * np.max(q_table[new_state,:]))

        state = new_state
        rewards_current_episode += reward

        if done == True:
            break

    #we decay the exploration rate as we want to converge as we reach the optiml policy
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate * episode)

    rewards_all_episodes.append(rewards_current_episode)

print("done")


done


In [14]:
q_table

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 6.97792437,  7.02739562,  6.46212172,  7.03828241,  9.6220697 ,
        -1.14296518],
       [11.23334491, 12.73665221, 11.29024394, 12.79130198, 14.11880599,
         3.77801419],
       ...,
       [ 8.54572936, 15.24697999,  7.95043066,  7.47770136, -0.70922442,
        -2.43491602],
       [ 6.95815047,  7.58293539,  4.86600791, 10.66927363, -2.6796169 ,
        -3.33493812],
       [14.18993183, 13.62010357, 14.49512679, 18.79836696,  7.56526216,
         4.6644392 ]])

In [15]:
#Calculate and print average reward per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes/1000) 
count = 1000
print("Average per thousand episodes") 
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count+= 1000




Average per thousand episodes
1000 :  -291.8100000000003
2000 :  8.27199999999996
3000 :  14.06999999999986
4000 :  15.067999999999854
5000 :  14.940999999999848
6000 :  -142.7459999999999
7000 :  8.601999999999945
8000 :  14.318999999999848
9000 :  14.832999999999876
10000 :  14.81399999999986


In [21]:
import time
#Visualize Agent
for episode in range(3):
       
    state = env.reset() 
    done = False 
    print("Episode is: " + str(episode))
    time.sleep(1)

    for step in range (max_steps_per_episode):
        clear_output(wait=True)

        env.render() 
        time.sleep(0.4)
        #now since the model is trained, take the max action.
        action = np.argmax (q_table[state,:])
        new_state, reward, done, info = env.step(action)
        
        if done:
            clear_output(wait=True)

            env.render() 
            if reward == 1:
                print("*****Reached Goal****")
                time.sleep(1)

                clear_output(wait=True)

            else:
                print("*****Failed*****")
                time.sleep(1)

                clear_output(wait=True)

            break


        state = new_state

env.close()



+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
**** *Failed*****


SO, as the agent learns for over 10000 episodes, it will learn teh desired performance.